In [1]:
import os
from typing import Optional
from collections import namedtuple, deque
import math
import random
import logging

import tqdm
import tqdm.notebook as tqdm_notebook
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import IterableDataset
from torchmetrics import MeanMetric

from lightning.pytorch import LightningModule, loggers as pl_loggers, callbacks as pl_callbacks
from lightning.pytorch.trainer import Trainer
from lightning.pytorch import seed_everything

from rl.model.module.ppo_module import PPOLightningModule

In [2]:
logger = logging.getLogger('lightning.pytorch')
logger.setLevel(logging.INFO)

seed = 42
seed_everything(seed)

env_type = 'CartPole-v1'
log_root = 'logs'

tb_logger = pl_loggers.TensorBoardLogger(save_dir=log_root, name=env_type)
model_cfg = dict(
    env_cfg=dict(
        type=env_type,
        render_mode="rgb_array"),
    data_cfg=dict(
        total_timestep_n=2048,
        sample_timestep_n=1024,
        batch_size=16),
    ppo_cfg=dict(
        actor_cfg=dict(
            net_cfg=dict(
                channels=[64, 64],
                use_layer_norm=True,
                act_func='ReLU'),
            use_mlp=True),
        critic_cfg=dict(
            net_cfg=dict(
                channels=[64, 64],
                use_layer_norm=True,
                act_func='ReLU'),
            use_mlp=True)),
    loss_cfg=dict(
        policy_loss=dict(clip_coef=0.2),
        value_loss=dict(
            clip_coef=0.2,
            clip_vloss=False,
            vf_coef=1.0),
        entropy_loss=dict(ent_coef=0.0)),
    optim_cfg=dict(lr=1e-4),
    running_cfg=dict(
        seed=42,
        log_root=tb_logger.log_dir,
        gamma=0.99,
        gae_lambda=0.95,
        normalize_advantages=False,
        update_interval=10,
        update_steps=100),
)
callbacks = [
    pl_callbacks.ModelCheckpoint(os.path.join(tb_logger.log_dir, 'checkpoints'),  monitor='val/episode_reward',
                                 filename=f'{env_type}' + '-epoch_{epoch:02d}-reward_{val/episode_reward:.2f}',
                                 mode='max', verbose=False, save_top_k=3, auto_insert_metric_name=False)]

model = PPOLightningModule(**model_cfg)
trainer = Trainer(max_steps=50000, logger=tb_logger, check_val_every_n_epoch=1, callbacks=callbacks)
trainer.fit(model)

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params | Mode 
------------------------------------------------------
0 | ppo_model      | PPOModule  | 9.7 K  | train
1 | avg_pg_loss    | MeanMetric | 0      | train
2 | avg_value_loss | MeanMetric | 0      | train
3 | avg_ent_loss   | MeanMetric | 0      | train
------------------------------------------------------
9.7 K     Trainable params
0         Non-trainable params
9.7 K     Total params
0.039     Total estimated model params size (MB)
28        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/yuhang09/miniforge3/envs/ppo/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/yuhang09/miniforge3/envs/ppo/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/yuhang09/miniforge3/envs/ppo/lib/python3.9/site-packages/lightning/pytorch/utilities/data.py:123: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=50000` reached.
